In [3]:
import requests
import lxml.html as lh
import pandas as pd

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [5]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [6]:
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal code
"
2:"Borough
"
3:"Neighborhood
"


In [7]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [8]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [9]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [10]:
df.head(10)

,Postal code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
5,M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n
6,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government\n
7,M8A\n,Not assigned\n,\n
8,M9A\n,Etobicoke\n,Islington Avenue\n
9,M1B\n,Scarborough\n,Malvern / Rouge\n


In [11]:
df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head(10)

,Postcode,Borough,Neighbourhood
0,\n,M1A\n,Not assigned\n
1,\n,M2A\n,Not assigned\n
2,Parkwoods\n,M3A\n,North York\n
3,Victoria Village\n,M4A\n,North York\n
4,Regent Park / Harbourfront\n,M5A\n,Downtown Toronto\n
5,Lawrence Manor / Lawrence Heights\n,M6A\n,North York\n
6,Queen's Park / Ontario Provincial Government\n,M7A\n,Downtown Toronto\n
7,\n,M8A\n,Not assigned\n
8,Islington Avenue\n,M9A\n,Etobicoke\n
9,Malvern / Rouge\n,M1B\n,Scarborough\n


In [12]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,,M1A,Not assigned
1,,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Regent Park / Harbourfront,M5A,Downtown Toronto


In [18]:
df.drop(df.index[df['Postcode'] == ' '], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postcode,Borough,Neighbourhood
0,Parkwoods,M3A,North York
1,Victoria Village,M4A,North York
2,Regent Park / Harbourfront,M5A,Downtown Toronto
3,Lawrence Manor / Lawrence Heights,M6A,North York
4,Queen's Park / Ontario Provincial Government,M7A,Downtown Toronto
5,Islington Avenue,M9A,Etobicoke
6,Malvern / Rouge,M1B,Scarborough
7,Don Mills,M3B,North York
8,Parkview Hill / Woodbine Gardens,M4B,East York
9,"Garden District, Ryerson",M5B,Downtown Toronto


In [19]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,Agincourt,M1S,Scarborough
1,Alderwood / Long Branch,M8W,Etobicoke
2,Bathurst Manor / Wilson Heights / Downsview No...,M3H,North York
3,Bayview Village,M2K,North York
4,Bedford Park / Lawrence Manor East,M5M,North York
5,Berczy Park,M5E,Downtown Toronto
6,Birch Cliff / Cliffside West,M1N,Scarborough
7,Brockton / Parkdale Village / Exhibition Place,M6K,West Toronto
8,Business reply mail Processing CentrE,M7Y,East Toronto
9,CN Tower / King and Spadina / Railway Lands / ...,M5V,Downtown Toronto


In [20]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [21]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [24]:
df.shape

(103, 3)

In [26]:
df.to_csv(r'df_can.csv')

In [27]:
df

,Postcode,Borough,Neighbourhood
0,Agincourt,M1S,Scarborough
1,Alderwood / Long Branch,M8W,Etobicoke
2,Bathurst Manor / Wilson Heights / Downsview No...,M3H,North York
3,Bayview Village,M2K,North York
4,Bedford Park / Lawrence Manor East,M5M,North York
...,...,...,...
98,Willowdale / Newtonbrook,M2M,North York
99,Woburn,M1G,Scarborough
100,Woodbine Heights,M4C,East York
101,York Mills / Silver Hills,M2L,North York
